## Import libraries and data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')
%matplotlib inline

In [2]:
df = pd.read_csv("D:/AMIR/Programming/Data Science/Sales Data Analysis/Online Retail.csv", encoding="latin", dtype={'CustomerID': str})
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom


- Just by looking at the first 5 rows of our table, we can understand the structure and datatypes present in our dataset. 
- We can notice that we will have to deal with time series data, integers and floats, and categorical, and text data.

## Exploratory data analysis

- Every data science project starts with EDA as we have to understand what do we have to deal with. 
- I divide EDA into 2 types: visual and numerical. Let's start with numerical as the simple pandas method .describe() gives us a lot of useful information.

### Quick statistical overview

In [3]:
df.describe()

,Quantity,UnitPrice
count,541909.000000,541909.000000
mean,9.552250,4.611114
std,218.081158,96.759853
min,-80995.000000,-11062.060000
25%,1.000000,1.250000
50%,3.000000,2.080000
75%,10.000000,4.130000
max,80995.000000,38970.000000


- Just a quick look at data with the .describe() method gives us a lot of space to think. 
- We see negative quantities and prices, and we can see that not all records have CustomerID data.
- we can also see that the majority of transactions are for quantities from 3 to 10 items, majority of items have prices up to 5 pounds.
- we have a bunch of huge outliers we will have to deal with later.

### Dealing with types

- .read_csv() method performs basic type check, but it doesn't do that perfectly. 
- That's why it is much better to deal with data types in our dataframe before any modifications to prevent additional difficulties. 
- Every pandas dataframe has an attribute .dtypes which will help us understand what we currently have and what data has to be casted to correct types.

In [4]:
df.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID      object
Country         object
dtype: object

- If we have DateTime data it's better to cast it to DateTime type. 
- We don't touch InvoiceNo for now as it seems like data in this column has not only numbers. 
- we saw just the first 5 rows, while pandas during import scanned all the data and found that the type here is not numerical.

In [5]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df = df.set_index('InvoiceDate')

In [6]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country
InvoiceDate,,,,,,,
2010-12-01 08:26:00,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,17850,United Kingdom
2010-12-01 08:26:00,536365,71053,WHITE METAL LANTERN,6,3.39,17850,United Kingdom
2010-12-01 08:26:00,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75,17850,United Kingdom
2010-12-01 08:26:00,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,17850,United Kingdom
2010-12-01 08:26:00,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,17850,United Kingdom


###  Dealing with null values

- Next and very important step is dealing with missing values.
- Normally if you encounter null values in the dataset you have to understand nature of those null values and possible impact they could have on the model. 

- There are few strategies that we can use to fix our issue with null values:

    - delete rows with null values
    - delete the feature with null values
    - impute data with mean or median values or use another imputing strategy (method .fillna())


- Let's check out what we have here.

In [7]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

- CustomerID has too many null values and this feature cannot predict a lot so we can just drop it.
-  It could be reasonable to create another feature "Amount of orders per customer".

In [8]:
df = df.drop(columns=['CustomerID'])

Let's check out what kind of nulls we have in Description:

In [9]:
df[df['Description'].isnull()].head()

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,Country
InvoiceDate,,,,,,
2010-12-01 11:52:00,536414,22139,NaN,56,0.0,United Kingdom
2010-12-01 14:32:00,536545,21134,NaN,1,0.0,United Kingdom
2010-12-01 14:33:00,536546,22145,NaN,1,0.0,United Kingdom
2010-12-01 14:33:00,536547,37509,NaN,1,0.0,United Kingdom
2010-12-01 14:34:00,536549,85226A,NaN,1,0.0,United Kingdom


- The data in these rows is pretty strange as UnitPrice is 0, so these orders do not generate any sales. 
- We can impute it with "UNKNOWN ITEM" at the moment and deal with those later during the analysis.

In [10]:
df['Description'] = df['Description'].fillna('UNKNOWN ITEM')
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
UnitPrice      0
Country        0
dtype: int64